In [18]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import wfdb
import ast
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import *
from keras_radam import RAdam
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

# Data Preprocessing

In [19]:
# Due to memory constraints we can't load in entire dataset at once so we split into two halves
data = pd.read_csv("ptbxl_database.csv")
halfway = len(data)//2
data1, data2 = data[:halfway], data[halfway:]
path = "/home/walt/ml/ECG/data/"
sample_rate = 500

In [20]:
# Converts ECG data from raw files into numpy array
def load_data(df, sample_rate, path):
    if sample_rate==100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, key in data], dtype=np.float32)
    return data

In [21]:
# Load in first half of training data
X = load_data(data1, sample_rate, path)

In [22]:
# Extract annotations
Y = pd.read_csv("ptbxl_database.csv", index_col="ecg_id")
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

In [23]:
# Load scp_statements.csv for combining diagnoses
desc = pd.read_csv("scp_statements.csv", index_col=0)
desc = desc[desc.diagnostic==1]

In [24]:
# Aggregating diagnoses (Converting diagnostic dictionaries into disease labels)
def combine_diagnoses(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in desc.index:
            tmp.append(desc.loc[key].diagnostic_class)
    return tmp
Y["diagnostic_superclass"] = Y.scp_codes.apply(combine_diagnoses)

In [25]:
# One hot encoding labels
labels = Y.diagnostic_superclass.values
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)
mlb.classes_

array(['CD', 'HYP', 'MI', 'NORM', 'STTC'], dtype=object)

In [26]:
# Train test split
X_train, X_valid, y_train, y_valid = train_test_split(X, labels[:halfway], test_size=0.1, random_state=69420)

print(f"Training Data Shape: {X_train.shape}",
     f"Train Labels Shape: {y_train.shape}",
     f"Test Data Shape: {X_valid.shape}",
     f"Test Labels Shape: {y_valid.shape}", sep="\n")

Training Data Shape: (9826, 5000, 12)
Train Labels Shape: (9826, 5)
Test Data Shape: (1092, 5000, 12)
Test Labels Shape: (1092, 5)


# Model Replication

#### Paper can be found here https://www.thelancet.com/journals/landig/article/PIIS2589-7500(20)30107-2/fulltext

In [27]:
class ZhuModel(tf.keras.Model):
    """Changes in architecture
    - Original Model: Conv filters = 64,128,256,512. 
    - This model: Conv filters = 32,64,128,256. 
    - Reason: Original model trained with ~400k samples. This dataset has ~22k samples so filter sizes of that size
    will likely result in overfitting. 
    
    - Original Model: Dense Layer sizes (512, 512, 21)
    - This Model: Dense Layer sizes (256, 256, 5)
    - Reason: Reduce parameters and reduce chance of overfitting. Also, original model was trained to detect 21 
    separate classes whereas this one is detecting 5 classes due to the dataset it is being trained on."""
    
    def __init__(self):
        super(ZhuModel, self).__init__()
       
        # Input block
        self.conv1A = Conv1D(64, (15), 2, input_shape=X.shape[1:])
        self.bn1A = BatchNormalization()
        self.act1A = Activation("relu")
        self.pool1 = MaxPool1D(pool_size=2, strides=2)
        
        # 4 CONV + IDEN blocks
        
        # Conv Block 1
        self.bl1conv1 = Conv1D(64, (15), 2)
        self.bl1bn1 = BatchNormalization()
        self.bl1act1 = Activation("relu")
        self.bl1conv2 = Conv1D(64, (15), 1, padding="same")
        self.bl1bn2 = BatchNormalization()
        self.bl1act2 = Activation("relu")
        
        self.bl1pool1 = MaxPool1D(2, strides=2)
        
        # Iden Block 1
        self.bl1conv3 = Conv1D(64, (15), 1, padding="same")
        self.bl1bn3 = BatchNormalization()
        self.bl1act3 = Activation("relu")
        
        # Conv Block 2
        self.bl2conv1 = Conv1D(128, (15), 2)
        self.bl2bn1 = BatchNormalization()
        self.bl2act1 = Activation("relu")
        self.bl2conv2 = Conv1D(128, (15), 1, padding="same")
        self.bl2bn2 = BatchNormalization()
        self.bl2act2 = Activation("relu")
        self.bl2pool1 = MaxPool1D(2, strides=2)
   
        # Iden Block 2
        self.bl2conv3 = Conv1D(128, (15), 1, padding="same")
        self.bl2bn3 = BatchNormalization()
        self.bl2act3 = Activation("relu")
        
        # Conv Block 3
        self.bl3conv1 = Conv1D(256, (15), 2)
        self.bl3bn1 = BatchNormalization()
        self.bl3act1 = Activation("relu")
        self.bl3conv2 = Conv1D(256, (15), 1, padding="same")
        self.bl3bn2 = BatchNormalization()
        self.bl3act2 = Activation("relu")
        
        self.bl3pool1 = MaxPool1D(2, strides=2)
        
        # Iden Block 3
        self.bl3conv3 = Conv1D(256, (15), 1, padding="same")
        self.bl3bn3 = BatchNormalization()
        self.bl3act3 = Activation("relu")
        
        # Conv Block 4
        self.bl4conv1 = Conv1D(512, (15), 2)
        self.bl4bn1 = BatchNormalization()
        self.bl4act1 = Activation("relu")
        self.bl4conv2 = Conv1D(512, (15), 1, padding="same")
        self.bl4bn2 = BatchNormalization()
        self.bl4act2 = Activation("relu")
        
        self.bl4pool1 = MaxPool1D(2, strides=2)
        
        # Iden Block 4
        self.bl4conv3 = Conv1D(512, (15), 1, padding="same")
        self.bl4bn3 = BatchNormalization()
        self.bl4act3 = Activation("relu")
        
        #self.pool2 = AveragePooling1D(2)
        self.flatten = Flatten()
        # DENSE BLOCKS
        
        # 2 Dense Blocks
        self.fc1 = Dense(512)
        self.fc1act = Activation("relu")
        self.drop1 = Dropout(0.6)
        self.fc2 = Dense(512)
        self.fc2act = Activation("relu")
        self.drop2 = Dropout(0.6)
        
        # Output layer
        self.L = Dense(5, activation="sigmoid")
        
    
    def call(self, inputs):
        # Input convolution
        x = self.conv1A(inputs)
        x = self.bn1A(x)
        x = self.act1A(x)
        x = self.pool1(x)
        
        # CONVBLOCK Pass 1
        orig = x
        orig = self.bl1conv1(orig)
        orig = self.bl1bn1(orig)
        
        x = self.bl1conv1(x)
        x = self.bl1bn1(x)
        x = self.bl1act1(x)
        x = self.bl1conv2(x)
        x = self.bl1bn2(x)
        x = self.bl1act2(x)
        
        # Add residual to original input
        x += orig
        x = self.bl1act3(x)
        x = self.bl1pool1(x)
        
        # IDENBLOCK pass 1
        orig = x
        x = self.bl1conv3(x)
        x = self.bl1bn3(x)
        x = self.bl1act3(x)
        x += orig
        x = self.bl1act3(x)
        
        # CONVBLOCK Pass 2
        orig = x
        orig = self.bl2conv1(orig)
        orig = self.bl2bn1(orig)
        
        x = self.bl2conv1(x)
        x = self.bl2bn1(x)
        x = self.bl2act1(x)
        x = self.bl2conv2(x)
        x = self.bl2bn2(x)
        x = self.bl2act2(x)
        
        # Add residual to original input
        x+=orig
        x = self.bl2act3(x)
        x = self.bl2pool1(x)
        
        # IDENBLOCK pass 2
        orig = x
        x = self.bl2conv3(x)
        x = self.bl2bn3(x)
        x = self.bl2act3(x)
        x += orig
        x = self.bl2act3(x)

        # CONVBLOCK Pass 3
        orig = x
        orig = self.bl3conv1(orig)
        orig = self.bl3bn1(orig)
        
        x = self.bl3conv1(x)
        x = self.bl3bn1(x)
        x = self.bl3act1(x)
        x = self.bl3conv2(x)
        x = self.bl3bn2(x)
        x = self.bl3act2(x)
        
        # Add residual to original input
        x+=orig
        x = self.bl3act3(x)
        x = self.bl3pool1(x)
        
        # IDENBLOCK pass 3
        orig = x
        x = self.bl3conv3(x)
        x = self.bl3bn3(x)
        x = self.bl3act3(x)
        x += orig
        x = self.bl3act3(x)

        # CONVBLOCK Pass 4
        orig = x
        orig = self.bl4conv1(orig)
        orig = self.bl4bn1(orig)
        
        x = self.bl4conv1(x)
        x = self.bl4bn1(x)
        x = self.bl4act1(x)
        x = self.bl4conv2(x)
        x = self.bl4bn2(x)
        x = self.bl4act2(x)
        
        # Add residual to original input
        x += orig
        x = self.bl4act3(x)
        #x = self.bl4pool1(x)
        
        # IDENBLOCK pass 4
        orig = x
        x = self.bl4conv3(x)
        x = self.bl4bn3(x)
        x = self.bl4act3(x)
        x += orig
        x = self.bl4act3(x)
        
        #x = self.pool2(x)
        x = self.flatten(x)
        
        # Fully Connected Layers
        x = self.fc1(x)
        x = self.fc1act(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.fc2act(x)
        x = self.drop2(x)
        
        # Output Layer
        x = self.L(x)
        
        return x

In [28]:
# Calculating class weights
norm, cd, hyp, mi, sttc = [],[],[],[],[]
def count_classes(x):
    
    if "NORM" in x: 
        norm.append(x)
    if "CD" in x:
        cd.append(x)
    if "HYP" in x:
        hyp.append(x)
    if "MI" in x:
        mi.append(x)
    if "STTC" in x:
        sttc.append(x)
     
Y.diagnostic_superclass.apply(lambda x: count_classes(x))    
print(f"Norm: {len(norm)}", f"CD: {len(cd)}", f"hyp {len(hyp)}", f"MI: {len(mi)}", f"STTC: {len(sttc)}", sep="\n")
weights = [1 - len(cls)/len(data) for cls in list([norm, cd, hyp, mi, sttc])]
class_weights = {
    0: weights[0],
    1: weights[1],
    2: weights[2],
    3: weights[3],
    4: weights[4]
}
class_weights

# Creating learning rate schedule
def exponential_decay(init_lr, s):
    def exponential_decay_fn(epoch):
        return init_lr * 0.1**(epoch/s)
    return exponential_decay_fn

exp_decay_fn = exponential_decay(0.01, 20)
lr_schedule = tf.keras.callbacks.LearningRateScheduler(exp_decay_fn)

# Training parameters
EPOCHS = 100
BATCH = 64

Norm: 9528
CD: 4907
hyp 2655
MI: 5486
STTC: 5250


In [29]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, verbose=1)
early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0.005, patience=8)
zhu = ZhuModel()
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
zhu.compile(optimizer=opt, loss="binary_crossentropy", metrics=["categorical_accuracy"])

In [ ]:
H = zhu.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=BATCH, epochs=EPOCHS, 
            callbacks=[lr_schedule, early_stop], class_weight=class_weights)

Epoch 1/100
154/154 [==============================] - 15s 100ms/step - loss: 0.4040 - categorical_accuracy: 0.4455 - val_loss: 0.4694 - val_categorical_accuracy: 0.4982 - lr: 0.0100
Epoch 2/100
154/154 [==============================] - 14s 94ms/step - loss: 0.3098 - categorical_accuracy: 0.4814 - val_loss: 0.4613 - val_categorical_accuracy: 0.4863 - lr: 0.0089
Epoch 3/100
154/154 [==============================] - 14s 94ms/step - loss: 0.3004 - categorical_accuracy: 0.4894 - val_loss: 0.4195 - val_categorical_accuracy: 0.4918 - lr: 0.0079
Epoch 4/100
154/154 [==============================] - 14s 94ms/step - loss: 0.2888 - categorical_accuracy: 0.4957 - val_loss: 0.4060 - val_categorical_accuracy: 0.4936 - lr: 0.0071
Epoch 5/100
154/154 [==============================] - 14s 94ms/step - loss: 0.2844 - categorical_accuracy: 0.5056 - val_loss: 0.4049 - val_categorical_accuracy: 0.4808 - lr: 0.0063
Epoch 6/100
154/154 [==============================] - 15s 95ms/step - loss: 0.2812 - cat

In [16]:
# Saving model after training first half
tf.keras.models.save_model(zhu, "zhu_first1", save_format="tf")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: zhu_first1/assets


In [8]:
# Restarted notebook to load in second half of data so need to reload the model as well
X = load_data(data2, sample_rate, path)

In [9]:
zhu_final = tf.keras.models.load_model("zhu_first1")
zhu_final.optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

In [10]:
# Train test split
X_train, X_valid, y_train, y_valid = train_test_split(X, labels[halfway:], test_size=0.1, random_state=69420)

print(f"Training Data Shape: {X_train.shape}",
     f"Train Labels Shape: {y_train.shape}",
     f"Test Data Shape: {X_valid.shape}",
     f"Test Labels Shape: {y_valid.shape}", sep="\n")

Training Data Shape: (9827, 5000, 12)
Train Labels Shape: (9827, 5)
Test Data Shape: (1092, 5000, 12)
Test Labels Shape: (1092, 5)


In [14]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5, factor=0.1, verbose=1)
stop = tf.keras.callbacks.EarlyStopping(patience=10, monitor="val_loss", verbose=1)
zhu_final.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=EPOCHS, 
              batch_size=BATCH, class_weight=class_weights, callbacks=[reduce_lr, stop])

Epoch 1/100
154/154 [==============================] - 15s 99ms/step - loss: 0.2323 - categorical_accuracy: 0.6827 - val_loss: 0.3082 - val_categorical_accuracy: 0.6410 - lr: 1.0000e-04
Epoch 2/100
154/154 [==============================] - 14s 93ms/step - loss: 0.2060 - categorical_accuracy: 0.7022 - val_loss: 0.2915 - val_categorical_accuracy: 0.6658 - lr: 1.0000e-04
Epoch 3/100
154/154 [==============================] - 14s 94ms/step - loss: 0.1984 - categorical_accuracy: 0.7104 - val_loss: 0.2879 - val_categorical_accuracy: 0.6822 - lr: 1.0000e-04
Epoch 4/100
154/154 [==============================] - 14s 94ms/step - loss: 0.1893 - categorical_accuracy: 0.7227 - val_loss: 0.2931 - val_categorical_accuracy: 0.6603 - lr: 1.0000e-04
Epoch 5/100
154/154 [==============================] - 14s 94ms/step - loss: 0.1818 - categorical_accuracy: 0.7255 - val_loss: 0.2920 - val_categorical_accuracy: 0.6777 - lr: 1.0000e-04
Epoch 6/100
154/154 [==============================] - 15s 96ms/step -